<a href="https://colab.research.google.com/github/Cosmo571/Self_evolving_nn/blob/main/Self_Evolving_Neural_Networks_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prep:


 Main functions: - algorithm

 -function that returns the GC potential

 -function that evolves the network
 
 Helper functions: -function that evolves the network needs a creator function that creates new networks in tensorflow.


In [ ]:
# Libraries
import tensorflow as tf
import numpy as np
import time
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import model_from_json
from sklearn import datasets
from sklearn.model_selection import train_test_split
from itertools import product
from collections.abc import Iterable

In [ ]:
# Functions
def add_neuron(model: keras.Model, chosen_layer: int) -> "keras.Model":
  model.layers[chosen_layer].units += 1
  n_model = model_from_json(model.to_json())
  return n_model
def remove_neuron(model: keras.Model, chosen_layer: int) -> "keras.Model":
  model.layers[chosen_layer].units -= 1
  n_model = model_from_json(model.to_json())
  return n_model
def add_layer(model: keras.Model, position: int, neurons: int) -> "keras.Model": 
  '''
  Adds a relu layer to a keras.Sequential model. 
  Note: The implementation is a bit hacky and I am not satisfied with it. Try to come up with other solutions
  '''
  layer_list = [layr.units for layr in model.layers]
  #take the TensorFlow dimension, flips it, removes the None values, changes it into a tuple. This is done as in not to result in a multiple output shapes error (underdefined model)
  input = tuple(filter(lambda item: item is not None, np.flip(model.input.shape)))
  last_layer = layers.Dense(1, activation='sigmoid') # This is optimised for binary classification problems
  layer_list.insert(position, neurons)
  layer_list.pop()
  n_model = keras.Sequential()
  n_model.add(layers.Input(shape=input))
  for neur in layer_list:
    n_model.add(layers.Dense(units=neur, activation='relu'))
  n_model.add(last_layer)
  return n_model
def remove_layer(model: keras.Model, position: int) -> "keras.Model":
  '''
  Removes a layer in a similar fashion to add_layer()
  '''
  layer_list = [layr.units for layr in model.layers]
  #take the TensorFlow dimension, flips it, removes the None values, changes it into a tuple. This is done as in not to result in a multiple output shapes error (underdefined model)
  input = tuple(filter(lambda item: item is not None, np.flip(model.input.shape)))
  last_layer = layers.Dense(1, activation='sigmoid') # This is optimised for binary classification problems
  layer_list.pop(position)
  layer_list.pop()
  n_model = keras.Sequential()
  n_model.add(layers.Input(shape=input))
  for layr in layer_list:
    n_model.add(layers.Dense(units=layr, activation='relu'))
  n_model.add(last_layer)
  return n_model  
def remove_layer_experimental(model: keras.Model, position: int, neurons: int) -> "keras.Model":
  '''
  I like this version of remove layer more. It's more versatile. However, I can see it causing plenty of bugs.
  '''
  layer_list = model.layers
  #take the TensorFlow dimension, flips it, removes the None values, changes it into a tuple. This is done as in not to result in a multiple output shapes error (underdefined model)
  input = tuple(filter(lambda item: item is not None, np.flip(model.input.shape)))
  layer_list.pop(position)
  n_model = keras.Sequential()
  n_model.add(layers.Input(shape=input))
  for layer in layer_list:
    n_model.add(layer)
  return n_model
def calculate_neuron_acceptance_probability( NN:float, energy_N:float, energy_N_1, beta:float , nu:float) -> "float":
  coeff = 1/(NN+1)
  z = np.exp(-beta*nu)
  exponent  = np.exp(-beta*(energy_N_1-energy_N))
  probability = coeff * exponent * z
  if(probability > 1):
    probability = 1
  return probability 
def calculate_neuron_removal_probability(NN:float, energy_N:float, energy_N_1, beta:float , nu:float) -> "float":
  coeff = NN
  inverse_z = np.exp(beta*nu)
  exponent  = np.exp(-beta*(energy_N-energy_N_1))
  probability = coeff * exponent * inverse_z
  if(probability > 1):
    probability = 1
  return probability 
def calculate_layer_acceptance_probability(NL: float, M:float, energy_NL:float, energy_NL_1, beta:float , nu:float) -> "float":
  coeff = (NL+M) / (NL+1)
  z = np.exp(-beta*nu*M)
  exponent = np.exp(-beta*(energy_NL_1-energy_NL))
  probability = coeff * exponent * z
  if(probability > 1):
    probability = 1
  return probability
def calculate_layer_removal_probability(NL: float, M:float, energy_NL:float, energy_NL_1, beta:float , nu:float) -> "float":
  coeff = NL / (M*(NL-1))
  inverse_z = np.exp(beta*nu*M)
  exponent = np.exp(-beta*(energy_NL-energy_NL_1))
  probability = coeff * exponent * inverse_z
  if(probability > 1):
    probability = 1
  return probability
def construct_model_binary_classification(input_shape: tuple, layer_list) -> "keras.Model":
  '''
  Adapt the last layer to anything
  '''
  model = keras.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for neurons in layer_list: 
    model.add(keras.layers.Dense(neurons, activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  return model

In [ ]:
# run simulation functions
# Unstable - doesn't work for some reason. I'll look more into it. initially I had iterables (x_train, y_train) as : Interable in annotations. Still doesn't work
# Always run the next cell instead
def save_model(model: keras.Model) -> "None":
  '''
  Saves model in the filepath stated in the initial name declaration. 
  The naming rule is: layer_neurons.nextLayer_nextLayerNeurons
  '''
  name="/Projects/models_sim_date/"
  for i in range(len(model.layers)):
    name += "l_" + str(i+1) + "_n_" +str(model.layers[i].units)
    name += "."
  #name += "t_" + str(time.time()) Removed at the moment to shorten the filenames
  model.save(name)
  print('saved ' + name + "/")
  return
def run_model(model: keras.Model, x_train, y_train, x_test, y_test) -> "float":
  opt = keras.optimizers.Adam(learning_rate=0.01)
  model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=opt)
  model.fit(x_train, y_train, epochs=5, batch_size=5,verbose=0)
  loss , mae = model.evaluate(x_test, y_test)
  return loss

def run_sim_neuron(model: keras.Model, loss: float, x_train, y_train, x_test, y_test, beta: float, nu: float) -> "keras.Model":
  '''
  function that runs move 1. loss was the term chosen for the energy of the system before the move, energy was chosen for the energy fo the system after the move.
  even if they are technically the same in terms of units (both losses/energy)
  this was chosen to avoid confusion between the two
  '''
  #brute force functions to solve the neuron removal problem until I ask stefano
  check_neurons_list = [lay.units <= 1 for lay in model.layers[1:-1]]
  if all(check_neurons_list) == True:
    return model
  while True: 
    layer = np.random.randint(low=1, high=len(model.layers)-1)
    NN = model.layers[layer].units
    if NN > 1: 
      break
  #END BRUTE FORCE
  add_neuron_prob = np.random.choice([True, False])
  if add_neuron_prob == True: 
    new_model = add_neuron(model, layer)
  else:
    new_model = remove_neuron(model, layer)
  #calculate model loss 
  energy = run_model(new_model, x_train, y_train, x_test, y_test)
  #run acceptance
  if(add_neuron_prob == True):
    probability = calculate_neuron_acceptance_probability(NN, loss, energy, beta, nu)
  else:
    probability = calculate_neuron_removal_probability(NN-1, energy, loss, beta, nu)
    # it is assumed that for this NN+1(i) is the model before the operation, and NN(i) is the model after the operation, hence the NN-1, and the reversed orders of energy and loss.
  if np.random.rand() < probability:
    print('model saved '+ str(probability) + " " + str(loss) + " " + str(energy))
    #just for trial, we save every model we ever make for this time
    save_model(new_model)
    return new_model
  else:
    print('model failed '+ str(probability) + " " + str(loss) + " " + str(energy))
    return model

def run_sim_layer(model: keras.Model, loss: float, x_train, y_train, x_test, y_test, beta: float, nu: float) -> "keras.Model":
  '''
  function that runs move 1. loss was the term chosen for the energy of the system before the move, energy was chosen for the energy fo the system after the move.
  even if they are technically the same in terms of units (both losses/energy)
  this was chosen to avoid confusion between the two
  '''
  neurons = np.random.randint(low=1, high=M)
  add_layer_prob = np.random.choice([True, False])
  if add_layer_prob == True: 
    layer = np.random.randint(low=1, high=len(model.layers)-1)
    new_model = add_layer(model, layer, neurons)
  else:
    #brute force way
    if(len(model.layers)-2 <= 1):
      return model
    layer = np.random.randint(low=1, high=len(model.layers)-2)
    new_model = remove_layer(model, layer)
  #calculate model loss 
  NL = len(new_model.layers)
  energy = run_model(new_model, x_train, y_train, x_test, y_test)
  #run acceptance
  if(add_layer_prob == True):
    probability = calculate_layer_acceptance_probability(NL, neurons, loss, energy, beta, nu)
  else:
    probability = calculate_layer_removal_probability(NL, M, energy, loss, beta, nu)
    # it is assumed that for this NN+1(i) is the model before the operation, and NN(i) is the model after the operation, hence the NN-1, and the reversed orders of energy and loss.
  if np.random.rand() < probability:
    print('model saved '+ str(probability) + " " + str(loss) + " " + str(energy))
    #just for trial, we save every model we ever make for this time
    save_model(new_model)
    return new_model
  else:
    print('model failed '+ str(probability) + " " + str(loss) + " " + str(energy))
    return model

def run_sim_step(model: keras.Model, x_train, y_train, x_test, y_test, f: float, beta: float, nu: float, M: int) -> "keras.Model":
  print('check step')
  loss = run_model(model, x_train, y_train, x_test, y_test)
  print('loss is checked')
  if np.random.rand() < f:
    print('start neuron')
    model = run_sim_neuron(model, loss, x_train, y_train, x_test, y_test, beta, nu)
    print('check step neuron')
  else:
    print('start layer')
    model = run_sim_layer(model, loss, x_train, y_train, x_test, y_test, beta, nu, M)
    print('check step layer')
  return model 

def run_sim_for_loop(x_train, y_train, x_test, y_test, f: float, beta: float, nu: float, M: int, number_of_starting_layers: int, low_neuron: int, high_neuron: int, iterations: int) -> keras.Model:
  print('check loop')
  model_layers = np.random.randint(low=low_neuron, high=high_neuron, size=number_of_starting_layers)
  model = construct_model_binary_classification(x_train.shape, model_layers)
  loss = run_model(model, x_train, y_train, x_test, y_test)
  for i in range(iterations):
    model = run_sim_step(model, x_train, y_train, x_test, y_test, f, beta, nu, M)
  return model


In [ ]:
# run simulation functions
def save_model(model):
  name="/Projects/models_sim_0_21.08.2022/"
  for i in range(len(model.layers)):
    name += "l_" + str(i) + "_n_" +str(model.layers[i].units)
    name += "__"
  #name += "t_" + str(time.time())
  model.save(name)
  print('saved ' + name + "/")
  return
def run_model(model, x_train, y_train, x_test, y_test):
  opt = keras.optimizers.Adam(learning_rate=0.01)
  model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=opt)
  model.fit(x_train, y_train, epochs=5, batch_size=5,verbose=0)
  loss , mae = model.evaluate(x_test, y_test)
  return loss

def run_sim_neuron(model, loss, x_train, y_train, x_test, y_test, beta, nu):
  '''
  function that runs move 1. loss was the term chosen for the energy of the system before the move, energy was chosen for the energy fo the system after the move.
  even if they are technically the same in terms of units (both losses/energy)
  this was chosen to avoid confusion between the two
  '''
  #brute force functions to solve the neuron removal problem until I ask stefano
  check_neurons_list = [lay.units <= 1 for lay in model.layers[1:-1]]
  if all(check_neurons_list) == True:
    return model
  while True: 
    layer = np.random.randint(low=1, high=len(model.layers)-1)
    NN = model.layers[layer].units
    if NN > 1: 
      break
  #END BRUTE FORCE
  add_neuron_prob = np.random.choice([True, False])
  if add_neuron_prob == True: 
    new_model = add_neuron(model, layer)
  else:
    new_model = remove_neuron(model, layer)
  #calculate model loss 
  energy = run_model(new_model, x_train, y_train, x_test, y_test)
  #run acceptance
  if(add_neuron_prob == True):
    probability = calculate_neuron_acceptance_probability(NN, loss, energy, beta, nu)
  else:
    probability = calculate_neuron_removal_probability(NN-1, energy, loss, beta, nu)
    # it is assumed that for this NN+1(i) is the model before the operation, and NN(i) is the model after the operation, hence the NN-1, and the reversed orders of energy and loss.
  if np.random.rand() < probability:
    print('model saved '+ str(probability) + " " + str(loss) + " " + str(energy))
    #just for trial, we save every model we ever make for this time
    save_model(new_model)
    return new_model
  else:
    print('model failed '+ str(probability) + " " + str(loss) + " " + str(energy))
    return model
  #just to be sure
  #return model

def run_sim_layer(model, loss, x_train, y_train, x_test, y_test, beta, nu, M):
  '''
  function that runs move 1. loss was the term chosen for the energy of the system before the move, energy was chosen for the energy fo the system after the move.
  even if they are technically the same in terms of units (both losses/energy)
  this was chosen to avoid confusion between the two
  '''
  neurons = np.random.randint(low=1, high=M)
  add_layer_prob = np.random.choice([True, False])
  if add_layer_prob == True: 
    layer = np.random.randint(low=1, high=len(model.layers)-1)
    new_model = add_layer(model, layer, neurons)
  else:
    #brute force way
    if(len(model.layers)-2 <= 1):
      return model
    layer = np.random.randint(low=1, high=len(model.layers)-2)
    new_model = remove_layer(model, layer)
  #calculate model loss 
  NL = len(new_model.layers)
  energy = run_model(new_model, x_train, y_train, x_test, y_test)
  #run acceptance
  if(add_layer_prob == True):
    probability = calculate_layer_acceptance_probability(NL, neurons, loss, energy, beta, nu)
  else:
    probability = calculate_layer_removal_probability(NL, M, energy, loss, beta, nu)
    # it is assumed that for this NN+1(i) is the model before the operation, and NN(i) is the model after the operation, hence the NN-1, and the reversed orders of energy and loss.
  if np.random.rand() < probability:
    print('model saved '+ str(probability) + " " + str(loss) + " " + str(energy))
    #just for trial, we save every model we ever make for this time
    save_model(new_model)
    return new_model
  else:
    print('model failed '+ str(probability) + " " + str(loss) + " " + str(energy))
    return model
  #just to be sure
  #return model

def run_sim_step(model, x_train, y_train, x_test, y_test, f, beta, nu, M):
  print('check step')
  loss = run_model(model, x_train, y_train, x_test, y_test)
  print('loss is checked')
  if np.random.rand() < f:
    print('start neuron')
    model = run_sim_neuron(model, loss, x_train, y_train, x_test, y_test, beta, nu)
    print('check step neuron')
  else:
    print('start layer')
    model = run_sim_layer(model, loss, x_train, y_train, x_test, y_test, beta, nu, M)
    print('check step layer')
  return model 

def run_sim_for_loop(x_train, y_train, x_test, y_test, f, beta, nu, M, number_of_starting_layers, low_neuron, high_neuron, iterations):
  print('check loop')
  model_layers = np.random.randint(low=low_neuron, high=high_neuron, size=number_of_starting_layers)
  model = construct_model_binary_classification(x_train.shape[1], model_layers)
  loss = run_model(model, x_train, y_train, x_test, y_test)
  for i in range(iterations):
    model = run_sim_step(model, x_train, y_train, x_test, y_test, f, beta, nu, M)
  return model

In [ ]:
# Apply here
x, y = datasets.make_moons(n_samples=1000, shuffle=True, noise=0.2, random_state=None)
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size = 0.2)
x_test, y_test = datasets.make_moons(n_samples=250, shuffle=True, noise=0.2, random_state=None)
mode = run_sim_for_loop(x_train, y_train, x_dev, y_dev, 0.7, 52, 0.01, 4, 3, 2, 8, 50)
mode.summary()
loss, mae = mode.evaluate(x_test, y_test)
print(loss, mae)

check loop
7/7 [==============================] - 0s 3ms/step - loss: 0.1638 - accuracy: 0.9450
check step
7/7 [==============================] - 0s 3ms/step - loss: 0.0982 - accuracy: 0.9650
loss is checked
start layer
7/7 [==============================] - 0s 3ms/step - loss: 0.6994 - accuracy: 0.4400
model failed 7.43167719289305e-15 0.09821504354476929 0.699382483959198
check step layer
check step
7/7 [==============================] - 0s 3ms/step - loss: 0.0901 - accuracy: 0.9750
loss is checked
start neuron
7/7 [==============================] - 0s 2ms/step - loss: 0.2733 - accuracy: 0.8850
model failed 7.229016536030759e-06 0.09011543542146683 0.27330097556114197
check step neuron
check step
7/7 [==============================] - 0s 3ms/step - loss: 0.1115 - accuracy: 0.9550
loss is checked
start neuron
7/7 [==============================] - 0s 3ms/step - loss: 0.6971 - accuracy: 0.4400
model failed 5.05922420692731e-15 0.11152983456850052 0.6971385478973389
check step neuron
ch

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
